In [2]:
import numpy as np

In [3]:
data_path = "../detection/data/acc_email.npy"

In [25]:
net = np.load(data_path, allow_pickle=True)
net.shape

(20,)

In [27]:
net[0]

<2029x2029 sparse matrix of type '<class 'numpy.float64'>'
	with 416 stored elements in Compressed Sparse Row format>

In [21]:
f = np.load("../detection/data/email0.01.npz")

train_pos_id, train_neg_id, test_pos_id, test_neg_id = (
    f["train_pos_id"],
    f["train_neg_id"],
    f["test_pos_id"],
    f["test_neg_id"],
)

train_pos, train_neg, test_pos, test_neg = (
    f["train_pos"],
    f["train_neg"],
    f["test_pos"],
    f["test_neg"],
)

In [19]:
train_pos_id.shape, train_neg_id.shape, test_pos_id.shape, test_neg_id.shape

((1546,), (1546,), (1227,), (10,))

In [24]:
train_pos.shape, train_neg.shape, test_pos.shape, test_neg.shape

((2, 1546), (2, 1546), (2, 1227), (2, 10))

In [29]:
train_pos

array([[ 330,  998,  477, ..., 1679, 1679, 1585],
       [ 546, 1257, 1668, ..., 1684, 1791, 1679]])

In [30]:
train_pos[0]

array([ 330,  998,  477, ..., 1679, 1679, 1585])

In [31]:
train_pos[1]

array([ 546, 1257, 1668, ..., 1684, 1791, 1679])

In [32]:
train_pos_id

array([ 5,  5,  5, ..., 13, 13, 13])